In [5]:
import os
import torch
import torchvision
import torch.utils.data as data_utils
from torchvision import datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms

In [6]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

In [7]:
n_epochs = 30 # Number of epochs
batch_size_train = 16 # Batch size for training
batch_size_test = 256 # Batch size for testing
learning_rate = 0.005 # Optimizier hyper-parameter
momentum = 0.7

In [8]:
transform = transforms.Compose(
    [torchvision.transforms.Resize(32),
     transforms.ToTensor(),
     torchvision.transforms.Normalize((0.5,), (0.5,))]
)


In [9]:
train_data = datasets.CIFAR100('dataset', train=True, download=True,transform=transform)
test_data = datasets.CIFAR100('dataset', train=False, download=True,transform=transform)

print('Train size: {}'.format(len(train_data)))
print('Test size: {}'.format(len(test_data)))

  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting dataset/cifar-100-python.tar.gz to dataset
Files already downloaded and verified
Train size: 50000
Test size: 10000


In [10]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size_train, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size_test, shuffle=True)

In [11]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.pool = nn.MaxPool2d(2,2)
        self.conv3 = nn.Conv2d(16,24,5)
        self.fc1 = nn.Linear(24, 180)
        self.fc2 = nn.Linear(180, 100)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = x.view(-1, 24)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return x


In [12]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = Net().to(device)
# Loss function
criterion = nn.CrossEntropyLoss()
# Optimizer
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)


In [13]:
# loop over the dataset multiple times
for epoch in range(n_epochs):
    print('Epoch: {0}'.format(epoch))
    running_loss = 0.0
    # loop over mini-batch
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # print statistics
        running_loss += loss.item()

# Save weights
print('Finished Training')
# change it to something like cifar.pth
PATH = './mnist_net.pth'
torch.save(net.state_dict(), PATH)

Epoch: 0
Epoch: 1
Epoch: 2
Epoch: 3
Epoch: 4
Epoch: 5
Epoch: 6
Epoch: 7
Epoch: 8
Epoch: 9
Epoch: 10
Epoch: 11
Epoch: 12
Epoch: 13
Epoch: 14
Epoch: 15
Epoch: 16
Epoch: 17
Epoch: 18
Epoch: 19
Epoch: 20
Epoch: 21
Epoch: 22
Epoch: 23
Epoch: 24
Epoch: 25
Epoch: 26
Epoch: 27
Epoch: 28
Epoch: 29
Finished Training


In [14]:
net = Net().to(device)
net.load_state_dict(torch.load(PATH))

correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        inputs, labels = data[0].to(device), data[1].to(device)
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Correct Predictions: {0}'.format(correct))
print('Total Predictions: {0}'.format(total))
print('Accuracy of the network on the 500 test images: %d %%' % (100 * correct / total))


Correct Predictions: 2933
Total Predictions: 10000
Accuracy of the network on the 500 test images: 29 %
